In [1]:
%load_ext autoreload
%autoreload 2
import os
os.sys.path.insert(0, '/home/ai/Documents/braindecode-master/braindecode')

# Cropped Decoding

Now we will use cropped decoding. Cropped decoding means the ConvNet is trained on time windows/time crops within the trials. Most of the code is identical to the [Trialwise Decoding Tutorial](TrialWise_Decoding.html), differences are explained in the text.

## Load data

In [2]:
import mne
from mne.io import concatenate_raws

# 5,6,7,10,13,14 are codes for executed and imagined hands/feet
subject_id = 1
event_codes = [5,6,9,10,13,14]

# This will download the files if you don't have them yet,
# and then return the paths to the files.
physionet_paths = mne.datasets.eegbci.load_data(subject_id, event_codes)

# Load each of the files
parts = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto', verbose='WARNING')
         for path in physionet_paths]

# Concatenate them
raw = concatenate_raws(parts)

# Find the events in this dataset
events = mne.find_events(raw, shortest_event=0, stim_channel='STI 014')

# Use only EEG channels
eeg_channel_inds = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# Extract trials, only using EEG channels
epoched = mne.Epochs(raw, events, dict(hands=2, feet=3), tmin=1, tmax=4.1, proj=False, picks=eeg_channel_inds,
                baseline=None, preload=True)


Trigger channel has a non-zero initial value of 1 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
179 events found
Event IDs: [1 2 3]
90 matching events found
No baseline correction applied
Not setting metadata
Loading data for 90 events and 497 original time points ...
0 bad epochs dropped


## Convert data to Braindecode format

In [3]:
import numpy as np
from braindecode.datautil.signal_target import SignalAndTarget
# Convert data from volt to millivolt
# Pytorch expects float32 for input and int64 for labels.
#print(events[:,1])
#print(events[:,2]-2)
#print(epoched.events[:,2]-2)
X = (epoched.get_data() * 1e6).astype(np.float32)
print(X.shape)
y = (epoched.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1
print(y.shape)

#X_train, X_test, y_train, y_test = X[:60],X[60:],y[:60],y[60:]

#print(X_train.shape)
#print(y_train.shape)
data = SignalAndTarget(X , y)
train_set = SignalAndTarget(X[:60], y=y[:60])
test_set = SignalAndTarget(X[60:], y=y[60:])

print(train_set)

(90, 64, 497)
(90,)


## Create the model

For cropped decoding, we now transform the model into a model that outputs a dense time series of predictions.
For this, we manually set the length of the final convolution layer to some length that makes the receptive field of the ConvNet smaller than the number of samples in a trial. Also, we use `to_dense_prediction_model`, which removes the strides in the ConvNet and instead uses dilated convolutions to get a dense output (see [Multi-Scale Context Aggregation by Dilated Convolutions](https://arxiv.org/abs/1511.07122) and our paper [Deep learning with convolutional neural networks for EEG decoding and visualization](https://arxiv.org/abs/1703.05051) Section 2.5.4 for some background on this).

In [4]:
from braindecode1.models.shallow_fbcsp_conv2 import ShallowFBCSPNet
from braindecode1.models.deep5 import Deep4Net
#from torch import nn
from braindecode.torch_ext.util import set_random_seeds
from braindecode.models.util import to_dense_prediction_model

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = False
set_random_seeds(seed=20170629, cuda=cuda)

# This will determine how many crops are processed in parallel
input_time_length = 450

n_classes = 2
print(train_set.X.shape[1])
in_chans = train_set.X.shape[1]
# final_conv_length determines the size of the receptive field of the ConvNet
model =  Deep4Net(in_chans=in_chans, n_classes=n_classes, input_time_length=input_time_length, stride_before_pool=True,
                        pool_time_stride=2,final_conv_length=1).create_network()

to_dense_prediction_model(model)
print(model)
if cuda:
    model.cuda()
    

from torch import optim

optimizer = optim.Adam(model.parameters())

64
Sequential(
  (dimshuffle): Expression(expression=_transpose_time_to_spat)
  (conv_time): Conv2d(1, 25, kernel_size=(10, 1), stride=(1, 1))
  (conv_spat): Conv2d(25, 25, kernel_size=(1, 64), stride=(1, 1), bias=False)
  (bnorm): BatchNorm2d(25, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (conv_nonlin): Expression(expression=elu)
  (pool): MaxPool2d(kernel_size=(3, 1), stride=(1, 1), padding=0, dilation=(2, 1), ceil_mode=False)
  (pool_nonlin): Expression(expression=identity)
  (drop_2): Dropout(p=0.5)
  (conv_2): Conv2d(25, 50, kernel_size=(10, 1), stride=(1, 1), dilation=(2, 1), bias=False)
  (bnorm_2): BatchNorm2d(50, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (nonlin_2): Expression(expression=elu)
  (pool_2): MaxPool2d(kernel_size=(3, 1), stride=(1, 1), padding=0, dilation=(4, 1), ceil_mode=False)
  (pool_nonlin_2): Expression(expression=identity)
  (drop_3): Dropout(p=0.5)
  (conv_3): Conv2d(50, 100, kernel_size=(10, 1), stride=(1, 1)

/home/dansy/Documents/braindecode-master/braindecode1/models/deep5.py:159: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(model.conv_time.weight, gain=1)
/home/dansy/Documents/braindecode-master/braindecode1/models/deep5.py:162: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(model.conv_time.bias, 0)
/home/dansy/Documents/braindecode-master/braindecode1/models/deep5.py:164: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(model.conv_spat.weight, gain=1)
/home/dansy/Documents/braindecode-master/braindecode1/models/deep5.py:168: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(model.bnorm.weight, 1)
/home/dansy/Documents/braindecode-master/braindecode1/models/deep5.py:169: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(mo

## Create cropped iterator

For extracting crops from the trials, Braindecode provides the  `CropsFromTrialsIterator?` class. This class needs to know the input time length of the inputs you put into the network and the number of predictions that the ConvNet will output per input. You can determine the number of predictions by passing dummy data through the ConvNet: 

In [5]:
from braindecode.torch_ext.util import np_to_var
# determine output size
test_input = np_to_var(np.ones((2, in_chans, input_time_length, 1), dtype=np.float32))
if cuda:
    test_input = test_input.cuda()
print(test_input.shape)
out = model(test_input)

n_preds_per_input = out.cpu().data.numpy().shape[2]
print("{:d} predictions per input/trial".format(n_preds_per_input))

torch.Size([2, 64, 450, 1])
47 predictions per input/trial


/home/dansy/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [6]:
from braindecode.datautil.iterators import CropsFromTrialsIterator
iterator = CropsFromTrialsIterator(batch_size=64,input_time_length=input_time_length,
                                  n_preds_per_input=n_preds_per_input)

print(iterator.get_batches(train_set, shuffle=False))

<generator object CropsFromTrialsIterator._yield_block_batches at 0x7fc30686f830>


The iterator has the method `get_batches`, which can be used to get randomly shuffled training batches with `shuffle=True` or ordered batches (i.e. first from trial 1, then from trial 2, etc.) with `shuffle=False`. Additionally, Braindecode provides the `compute_preds_per_trial_for_set` method, which accepts predictions from the ordered batches and returns predictions per trial. It removes any overlapping predictions, which occur if the number of predictions per input is not a divisor of the number of samples in a trial.


<div class="alert alert-info">

These methods can also work with trials of different lengths! For different-length trials, set `X` to be a list of 2d-arrays instead of a 3d-array.

</div>

## Training loop

The code below uses both the cropped iterator and the `compute_preds_per_trial_for_set` function to train and evaluate the network.

## K Folds cross validation

In [7]:
from sklearn.model_selection import KFold # import KFold for cross validation
from sklearn.datasets import make_classification
from skorch import NeuralNetClassifier

In [8]:
#kf = KFold(n_splits=6) # Define the split - into 3 folds 
kf = KFold(n_splits=5, random_state=7) #evaluate the k fold cross validation
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator
print(kf.get_n_splits(X)) 
print(kf) 

5
KFold(n_splits=5, random_state=7, shuffle=False)


In [9]:
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


TRAIN: [18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89] TEST: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89] TEST: [18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89] TEST: [36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 4

In [10]:
# reshaping the 3D array of the X dataset into a 2D array dataset for predictions 
                                                    #computation usage
Xtrain=X_train[:,:,496:]
Xtrain=np.squeeze(Xtrain)
print(Xtrain.shape)

Xtest=X_test[:,:,496:]
Xtest=np.squeeze(Xtest)
print(Xtest.shape)

print(len(y_train), len(Xtrain))

(72, 64)
(18, 64)
72 72


In [11]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold
from matplotlib import pyplot as plt

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

df = pd.DataFrame(np.squeeze(X[:,:,496:])) # reshaping the 3D array of the X dataset into a 2D array dataset for predictions 
Xdata =  np_to_var(np.squeeze(X[:,:,496:]))                                                  #computation usage
print(len(df))
# fit a model
#lm = linear_model.LinearRegression()

#model = lm.fit(Xtrain, y_train)
#predictions = lm.predict(Xtest)
#print(Xtrain.shape, y_train.shape, Xtest.shape)

#Kfolds Cross validation
for batch_X, batch_y in iterator.get_batches(train_set, shuffle=False):
    #pass
    for train_index, test_index in kf.split(batch_X):
       # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = batch_X[train_index], batch_X[test_index]
        y_train, y_test = batch_y[train_index], batch_y[test_index]


90


/home/dansy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
from braindecode.torch_ext.util import np_to_var, var_to_np
import torch.nn.functional as F
from numpy.random import RandomState
import torch as th
from braindecode.experiments.monitors import compute_preds_per_trial_for_set
rng = RandomState((2017,6,30))
accuracy_nb = {"Train": 0, "Test": 0}
accuracy_sum = {"Train": 0, "Test": 0}
accuracyF_nb = {"Train": 0, "Test": 0}
accuracyF_sum = {"Train": 0, "Test": 0}
i_trial_stops = [trial.shape[1] for trial in train_set.X]

for i_epoch in range(20):
    all_acc = []
    print("Epoch {:d}".format(i_epoch))
    fold=0
    for Train_index, Test_index in kf.split(train_set.X):
        fold= fold+1
        training_set = SignalAndTarget(train_set.X[train_index], y=y[train_index])
        val_set = SignalAndTarget(train_set.X[test_index], y=y[test_index])
        
        # Set model to training mode
        model.train()
        for batch_X, batch_y in iterator.get_batches(training_set, shuffle=False):
            net_in = np_to_var(batch_X)
            if cuda:
                net_in = net_in.cuda()
            net_target = np_to_var(batch_y)
            if cuda:
                net_target = net_target.cuda()
            # Remove gradients of last backward pass from all parameters 
            optimizer.zero_grad()
            outputs = model(net_in)
            # Mean predictions across trial
            # Note that this will give identical gradients to computing
            # a per-prediction loss (at least for the combination of log softmax activation 
            # and negative log likelihood loss which we are using here)
            outputs = th.mean(outputs, dim=2, keepdim=False)
            loss = F.nll_loss(outputs, net_target)
            loss.backward()
            optimizer.step()

        # Print some statistics each epoch
        model.eval()
        print("Fold {:d}".format(fold))
        for setname, dataset in (('Train', training_set),('Test', val_set)):
            
            # Collect all predictions and losses
            all_preds = []
            all_losses = []
            batch_sizes = []
            for batch_X, batch_y in iterator.get_batches(dataset, shuffle=False):
                net_in = np_to_var(batch_X)
                if cuda:
                    net_in = net_in.cuda()
                net_target = np_to_var(batch_y)
                if cuda:
                    net_target = net_target.cuda()
                outputs = model(net_in)
                all_preds.append(var_to_np(outputs))
                outputs = th.mean(outputs, dim=2, keepdim=False)
                loss = F.nll_loss(outputs, net_target)
                loss = float(var_to_np(loss))
                all_losses.append(loss)
                batch_sizes.append(len(batch_X))
            # Compute mean per-input loss 
            loss = np.mean(np.array(all_losses) * np.array(batch_sizes) /
                           np.mean(batch_sizes))
            print("{:6s} Loss: {:.5f}".format(setname, loss))
            # Assign the predictions to the trials
            preds_per_trial = compute_preds_per_trial_for_set(all_preds,
                                                              input_time_length,
                                                              dataset)
            # preds per trial are now trials x classes x timesteps/predictions
            # Now mean across timesteps for each trial to get per-trial predictions
            meaned_preds_per_trial = np.array([np.mean(p, axis=1) for p in preds_per_trial])
            predicted_labels = np.argmax(meaned_preds_per_trial, axis=1)
            accuracy = np.mean(predicted_labels == dataset.y)
            print("{:6s} Accuracy: {:.1f}%".format(
                setname, accuracy * 100))
            accuracyF_nb[setname] += 1
            accuracyF_sum[setname] += accuracy
            
            all_acc.append(accuracy * 100)
    for setname in ("Train", "Test"):
        print("{:6s} Mean fold Accuracy: {:.1f}%".format(
                    setname, accuracyF_sum[setname]*100/accuracyF_nb[setname]))
    for setname in ("Train", "Test"):        
        accuracy_nb[setname] += 1
        accuracy_sum[setname] += accuracyF_sum[setname]*100/accuracyF_nb[setname]
        print( "{} Number: {:.1f}%".format(setname, accuracy_nb[setname]))
for setname in ("Train", "Test"):
    print("{} Average accuracy: {:.1f}%".format(setname, accuracy_sum[setname]/accuracy_nb[setname]))

Epoch 0


/home/dansy/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Fold 1
Train  Loss: 4.04003
Train  Accuracy: 50.0%
Test   Loss: 4.46559
Test   Accuracy: 50.0%
Fold 2
Train  Loss: 0.75318
Train  Accuracy: 66.7%
Test   Loss: 0.77641
Test   Accuracy: 66.7%
Fold 3
Train  Loss: 0.38155
Train  Accuracy: 87.5%
Test   Loss: 1.02184
Test   Accuracy: 58.3%
Fold 4
Train  Loss: 1.54800
Train  Accuracy: 56.2%
Test   Loss: 2.63448
Test   Accuracy: 50.0%
Fold 5
Train  Loss: 1.41418
Train  Accuracy: 56.2%
Test   Loss: 2.86481
Test   Accuracy: 50.0%
Train  Mean fold Accuracy: 63.3%
Test   Mean fold Accuracy: 55.0%
Train Number: 1.0%
Test Number: 1.0%
Epoch 1
Fold 1
Train  Loss: 0.79339
Train  Accuracy: 77.1%
Test   Loss: 2.79560
Test   Accuracy: 41.7%
Fold 2
Train  Loss: 0.43974
Train  Accuracy: 85.4%
Test   Loss: 2.37649
Test   Accuracy: 41.7%
Fold 3
Train  Loss: 0.19674
Train  Accuracy: 91.7%
Test   Loss: 2.18873
Test   Accuracy: 50.0%
Fold 4
Train  Loss: 0.22475
Train  Accuracy: 93.8%
Test   Loss: 3.13166
Test   Accuracy: 50.0%
Fold 5
Train  Loss: 0.11644
Train 

Fold 1
Train  Loss: 0.00058
Train  Accuracy: 100.0%
Test   Loss: 4.14980
Test   Accuracy: 58.3%
Fold 2
Train  Loss: 0.00069
Train  Accuracy: 100.0%
Test   Loss: 4.19439
Test   Accuracy: 58.3%
Fold 3
Train  Loss: 0.00070
Train  Accuracy: 100.0%
Test   Loss: 4.17292
Test   Accuracy: 58.3%
Fold 4
Train  Loss: 0.00062
Train  Accuracy: 100.0%
Test   Loss: 4.16724
Test   Accuracy: 58.3%
Fold 5
Train  Loss: 0.00055
Train  Accuracy: 100.0%
Test   Loss: 4.14039
Test   Accuracy: 58.3%
Train  Mean fold Accuracy: 96.6%
Test   Mean fold Accuracy: 57.0%
Train Number: 15.0%
Test Number: 15.0%
Epoch 15
Fold 1
Train  Loss: 0.00073
Train  Accuracy: 100.0%
Test   Loss: 4.12341
Test   Accuracy: 58.3%
Fold 2
Train  Loss: 0.00074
Train  Accuracy: 100.0%
Test   Loss: 4.12677
Test   Accuracy: 58.3%
Fold 3
Train  Loss: 0.00091
Train  Accuracy: 100.0%
Test   Loss: 4.11695
Test   Accuracy: 58.3%
Fold 4
Train  Loss: 0.00099
Train  Accuracy: 100.0%
Test   Loss: 4.17848
Test   Accuracy: 58.3%
Fold 5
Train  Loss: 0.

In [13]:

from sklearn.model_selection import GridSearchCV


# create model
model_net = NeuralNetClassifier(
    module=model,
    max_epochs=20,
    batch_size=64,
    #criterion=th.nn.NLLLoss(np_to_var(batch_X), np_to_var(batch_y), ignore_index=-100, reduce=None),
    lr=0.1,
#     device='cuda',  # uncomment this to train with CUDA
)

model_net.initialize()
#model_net.initialize_module()
#print(model_net)
# fix random seed for reproducibility
seed = 7
# evaluate using 10-fold cross validation stratified cross val
kfold = StratifiedKFold(n_splits=5, random_state=seed)
print(np_to_var(X).size(), np_to_var(y).size() )

for batch_X, batch_y in iterator.get_batches(train_set, shuffle=False):
    pass
    #scores = cross_val_score(model_net, batch_X, batch_y, scoring="accuracy", cv=5)
    
    
#print(batch_X.type())
print(np_to_var(batch_X).type())
print("==================================")
#print(np_to_var(batch_X)[5].size() , np_to_var(batch_y).size())
print("==================================")
#print(np_to_var(batch_X[5]).dim())

param_grid = { 
    'param_1': [1, 2, 3, 4, 5],
    'param_2': [20, 40, 60],
    'param_3': [50, 100, 150]
}


cv_model = GridSearchCV(estimator=model_net, param_grid=param_grid,scoring="accuracy", cv=5)

#cv_model.fit(np_to_var(batch_X), np_to_var(batch_y))

A=batch_X
b=batch_y
model_net.fit(A,b)
#scores = cross_val_score(model_net, np_to_var(batch_X[0]), np_to_var(batch_y[0]), scoring="accuracy", cv=5)
#print ("Cross-validated scores:", scores)

torch.Size([90, 64, 497]) torch.Size([90])
torch.FloatTensor


/home/dansy/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


ValueError: Expected target size (48, 47), got torch.Size([48])

In [ ]:
# Make cross validated predictions
predictions = cross_val_predict(model_net, X, y, cv=6)
plt.scatter(y, predictions)

## Leave one out cross validation 

In [ ]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
loo.get_n_splits(X)


for train_index, test_index in loo.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(X_train, X_test, y_train, y_test)

## Dataset References


 This dataset was created and contributed to PhysioNet by the developers of the [BCI2000](http://www.schalklab.org/research/bci2000) instrumentation system, which they used in making these recordings. The system is described in:
 
     Schalk, G., McFarland, D.J., Hinterberger, T., Birbaumer, N., Wolpaw, J.R. (2004) BCI2000: A General-Purpose Brain-Computer Interface (BCI) System. IEEE TBME 51(6):1034-1043.

[PhysioBank](https://physionet.org/physiobank/) is a large and growing archive of well-characterized digital recordings of physiologic signals and related data for use by the biomedical research community and further described in:

    Goldberger AL, Amaral LAN, Glass L, Hausdorff JM, Ivanov PCh, Mark RG, Mietus JE, Moody GB, Peng C-K, Stanley HE. (2000) PhysioBank, PhysioToolkit, and PhysioNet: Components of a New Research Resource for Complex Physiologic Signals. Circulation 101(23):e215-e220.